In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from  ift6758.data.data_acquisition import Season


In [5]:
import pandas as pd
import numpy as np
def time_played(row):
    """
    return time in mins
    """
    # print(type(row['about.periodTime']))
    if row['result.event'] == 'GOAL' and row['about.period']>3:
        # return row['about.periodTime'] + row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        return float(min_sec1[0]) + float(min_sec1[1])/60
    else:
        # return row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        min_sec2 = row['about.periodTimeRemaining'].split(':')
        return float(min_sec1[0]) + float(min_sec2[0]) + (float(min_sec1[1]) +  float(min_sec2[1]))/60






In [6]:
# select_columns = ["result.event","gamePk","team.name","players","about.period","about.periodTime","about.periodType","about.periodTimeRemaining","coordinates.x","coordinates.y"]

def get_team_time(df_clean):
    sel_columns = ['gamePk','result.event','team.name','about.period','about.periodTime','about.periodTimeRemaining']
    df1 = df_clean[sel_columns].drop_duplicates() 
    df2 = df1.sort_values('about.periodTime', ascending=False).drop_duplicates(['gamePk','about.period'])
    df2['periodTimePlayed'] = df2.apply (lambda row: time_played(row), axis=1)
    df3 = df2.groupby('gamePk')['periodTimePlayed'].sum()
    df4 = df1[['gamePk','team.name']].drop_duplicates()
    df4['gameTime'] = df4.apply(lambda x: df3.get(key = x['gamePk']),axis=1)
    df5 = df4.groupby(['team.name'])['gameTime'].sum()
    total_time = df5.sum()
    return df5.to_dict(),total_time


    

time_game_dict = {}
for year in [2016,2017,2018,2019,2020]:
    season = Season(year,"../ift6758/data")
    df_clean = season.clean_data()
    time_game_dict[year] = {}
    time_game_dict[year]['team'],time_game_dict[year]['total_time'] = get_team_time(df_clean)
    time_game_dict[year]['teams_played'] = len(time_game_dict[year]['team'])





File already Exists, loading from ../ift6758/data/PICKLE//2016_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2017_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2018_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2019_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2020_clean.pkl


In [174]:
import numpy as np
def advanced_visualization(year=2018,team_name="Montréal Canadiens",bin_width=1):
# year=2018
# team_name="Montréal Canadiens"
    def bin_coordinates(row):
        """
        To explain the procedure of inverting the co-ordinates.
        """
        if row['coordinates.x'] <0:
            x_bin = -row['coordinates.x']
            y_bin= 42.5-row['coordinates.y']
        else:
            x_bin = row['coordinates.x']
            y_bin= 42.5+row['coordinates.y']
        
        
        return  np.floor(x_bin), np.floor(y_bin)
    def copyToOutput(row,out):
        """
        Setting out array
        """
        out[int(row['y_bin']),int(row['x_bin'])]=row['excess_shots']
        return "copiedToOutput"
    def sum_bin(out,bin_width):
        for j in range(0,out.shape[0],bin_width):
            for i in range(0,out.shape[1],bin_width):
                bin_sum=0
                for j1 in range(bin_width):
                    for i1 in range(bin_width):
                        if j+j1<out.shape[0] and i+i1<out.shape[1]:
                            bin_sum += out[j+j1][i+i1] 
                for j1 in range(bin_width):
                    for i1 in range(bin_width):
                        if j+j1<out.shape[0] and i+i1<out.shape[1]:
                            out[j+j1][i+i1] = bin_sum                

    season = Season(year,"../ift6758/data")
    df_clean = season.clean_data()
    rink_size = (85,100)
    out = np.zeros(rink_size)

    num_teams = df_clean['team.name'].unique().shape[0]
    print(f"num_teams {num_teams}")
        
    ## Reference: https://stackoverflow.com/a/52363890
    coordinates_df = df_clean.apply(lambda row: bin_coordinates(row),axis='columns', result_type='expand').rename({0: 'x_bin', 1: 'y_bin'}, axis=1)
    df_clean = pd.concat([df_clean, coordinates_df], axis='columns')
    num_games_by_team = df_clean.groupby('team.name')['gamePk'].count().to_dict()
    df_group1 = df_clean.groupby(['team.name','x_bin','y_bin'])['result.event'].count().reset_index()
    # df_group1['shots_average'] = df_group1['result.event']
    df_group1['shots_average'] = df_group1.apply(lambda r: r['result.event'] /num_games_by_team[r['team.name']],axis=1)
    df2 = df_group1.drop('result.event',axis=1).drop_duplicates()
    df_group2 = df2.groupby(['x_bin','y_bin'])['shots_average'].sum()/num_teams
    avg_shots_coordinate_dict = df_group2.to_dict()
    df2['excess_shots'] = df2.apply(lambda row: row['shots_average'] - avg_shots_coordinate_dict[(row['x_bin'],row['y_bin'])],axis=1)
    df3 = df2[df2['team.name']==team_name]

    df3.apply(lambda row: copyToOutput(row,out),axis=1)
    sum_bin(out,bin_width)
    print(df3[(df3['x_bin']==75.0)&(df3['y_bin']==36.0)])
    return out



In [175]:
out1=advanced_visualization(year=2017,team_name="San Jose Sharks",bin_width=1)

File already Exists, loading from ../ift6758/data/PICKLE//2017_clean.pkl
num_teams 35
             team.name  x_bin  y_bin  shots_average  excess_shots
46665  San Jose Sharks   75.0   36.0       0.000304     -0.001939


In [210]:
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Valid color strings are CSS colors, rgb or hex strings
# bottomfraction = 0-out1.min()/(out1.max()-out1.min())
colorscale = [[0, 'darkblue'],[0.5,'white'], [1, 'maroon']]

# out2 = out1*10000
out2=out1
# out2[(out2<0.1) & (out2>-0.1)]=0

f = gaussian_filter(out2, sigma=2)
# f=out2


# out1[out1]

IMG_PATH = "../figures/semi_nhl_rink.png"
fig = go.Figure()
# fig.update_yaxes(
#     scaleanchor = "x",
#     scaleratio = 1,
#   )
fig.update_layout(
    width=800,
    height=500,
)
fig.add_trace (
    go.Contour(
        z=f,
        # cmapk= 'magma',
        colorscale=colorscale,
        line_smoothing=1,
         opacity=0.7

        # zmin=-3,
        # zmid=0,
        # zmax=+3
    ))

# Add images
fig.add_layout_image(
        dict(
            source=IMG_PATH,
            xref="x2 domain",
            yref="y2 domain",
            x=0,
            y=1,
            sizex=1,
            sizey=1,
            opacity=0.5,
            sizing= "stretch",
            layer="below")
)

# Set templates
fig.update_layout(template="plotly_white")

fig.show()

In [219]:
import plotly.graph_objects as go
# Valid color strings are CSS colors, rgb or hex strings
# bottomfraction = 0-out1.min()/(out1.max()-out1.min())
colorscale = [[0, 'darkblue'], [0.48, 'white'],[0.52, 'white'], [1, 'maroon']]

fig = go.Figure(data =
    go.Contour(
        z=f1,
        # cmap = 'magma'
        # colorscale=colorscale,
        # zmin=-0.2,
        # zmid=0,
        # zmax=+0.2
    ))
# Update plot sizing
fig.update_layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white",
)

# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{"type": "contour"}],
                    label="contours",
                    method="restyle"
                ),
                dict(
                    args=[{"type":"contour"}],
                    label="Heatmap",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Trace type:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left")
    ]
)

fig.show()

# fig.show()